Transfer learning adalah sebuah teori yang mana kita menggunakan model yang sama dengan kasus yang berbeda untuk menghemat waktu. analoginya adalah, tentang orang yang lebih cepat mempelajari mengendarai motor dari orang yang sudah pernah mengendarai sepeda dibandingkan orang yang belum pernah sama sekali.

In [36]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, RMSprop, SGD
from keras.layers import Dense, MaxPool2D, Dropout, Conv2D, Flatten
from keras.models import Sequential
from keras.metrics import Recall, Precision
from keras.callbacks import EarlyStopping
import zipfile


In [37]:
with zipfile.ZipFile('D:\Kuliah\Materi\Orbit Campus\Materi\Computer Vision\Image Clasification using CNN\dataset_food.zip','r') as zip_ref:
    zip_ref.extractall('./')

In [38]:
#menentukan direktori dataset training dan validation

train_dir = './dataset_food/train/'
valid_dir = './dataset_food/valid/'

# menentukan ukuran gambar
img_width, img_height, channel = (224,224,3)

## Melakukan Data Generator
Kita melakukan data generator yang berisi perintah untuk melakukan data augmentasi, menentukan besar batch size, ukuran gambar, dan class mode.

In [39]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.2,
)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width,img_height),
    batch_size=16,
    class_mode='categorical'
)

Found 3949 images belonging to 5 classes.


In [40]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
)

valid_data = train_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_width,img_height),
    batch_size=16,
    class_mode='categorical'
)

Found 902 images belonging to 5 classes.


## Import Pre-Trained Model
kita akan menggunakan model VGG dan mobilenet yang telah disediakan oleh tensorflow

In [41]:
from keras.applications import VGG19
from keras.applications import MobileNet

base_model = VGG19(
    input_shape=(img_width,img_height,3),
    include_top=False,
    weights='imagenet'
)

for layer in base_model.layers:
    layer.trainable = False

base_model.summary()


Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

selanjutnya kita akan menambahkan beberapa dense layer dari pretarined model kita sesuai dengan kebutuhan kita. kali ini kita akan menambahkan 2 dense layer pada hidden layer dengan droput layer dan output layer berukuran 5(sesuai jumlah label) dengan aktivasi softmax.

In [42]:
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(5,activation='softmax')(x)

model = tf.keras.Model(inputs=base_model.input, outputs=x)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

## Compile Model
kita akan melakukan compile model dengan menggunakan optimizer adam dengan learning rate sebanyak 0,001 dan loss menggunakan categorical cross entropy.

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=[Recall(),Precision(),"accuracy"]
)

## Membuat callback dan fit model
kita akan membuat callback, pada sesi ini kita gunakan early stopping callback yang berfungsi untuk menghentikan prosses training jika suatu kondisi. kondisi yang kita buat pada callback kita kali ini adalah saat loss tidak ada penurunan pada 3 epoch terakhir. kemudian kita akan fit model dengan menggunakan epocs sebanyak 20.

In [46]:
model_callback = EarlyStopping(
    monitor='loss',
    verbose=1,
    mode='min',
    patience=3
)

history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=20,
    steps_per_epoch=50,
    callbacks=[model_callback],
    verbose=1
)

KeyboardInterrupt: 

## Evaluasi 
kita dapat mengecek apakah model kita overfitting, underfitting atau goodfitting menggunakan grafik berikut yang menunjukkan nilai akurasi disetiap epoch

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,3))
plt.plot(history.epoch,history.history['Accuracy'])
plt.plot(history.epoch,history.history['Val_Accuracy'])
plt.legend(['train loss','val loss'])
plt.title('Accuracy Diagram')
plt.xlabel('Epoch(s)')
plt.ylabel('Loss')
plt.show()

## Simpan Model

In [ ]:
model.save('myModel.h5')

## Load Model
kita akan menggunaakn model simpanan tadi untuk memprediksi sebuah gambar

In [ ]:
from keras.models import load_model
from keras.preprocessing import image

path_saved_model = 'myModel.h5'

loaded_model = load_model(path_saved_model)

## Predict

In [48]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from google.colab import files
from tensorflow.keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  img_path = '/content/' + fn
  img = image.load_img(img_path, target_size=(224, 224));
  img_array = image.img_to_array(img)
  img_array = np.expand_dims(img_array, axis=0)

  classes = loaded_model.predict(img_array/255);

  for i, class_ in enumerate(classes[0]):
      if class_ == classes[0].max():
          if i == 0:
              print("bakso")
          elif i == 1:
              print("gudeg")
          elif i == 2:
              print("gado")
          elif i == 3:
              print("rendang")
          else:
              print("sate")

ModuleNotFoundError: No module named 'google.colab'